## This file contains some of my failed attemps of further preprocessing, and could be used as a starting point for the future - for now, it should be omitted.

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

raw_data = pd.read_csv('data/processed/processedv1.csv')  # Placeholder for actual data loading
print(raw_data.shape)

(412698, 45)


/var/folders/pj/kpd4vj4j6_b_l77b3ry75hp40000gn/T/ipykernel_49058/1232175854.py:6: DtypeWarning: Columns (10,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data/processed/processedv1.csv')  # Placeholder for actual data loading


In [10]:
#similar analysis:
null_values_percentages = raw_data.isnull().sum()/412698 
null_values_nonzero = null_values_percentages[null_values_percentages > 0]
print("Missing values per column (ranked) (%):\n", null_values_nonzero.sort_values(ascending=False))
print("percentage of rows with a missing value:", (raw_data.isnull().any(axis=1).sum())/412698)

Missing values per column (ranked) (%):
 Hydraulics Flow              0.891264
Grouser Tracks               0.891264
Couple System                0.891024
Model Series                 0.857845
Steering Controls            0.826697
Differential Type            0.826595
Usage Band                   0.821492
Model Descriptor             0.818715
Backhoe Mounting             0.804428
Turbo Charged                0.803498
Version                      0.803498
Touchpad Type                0.803498
Blade Type                   0.801610
Travel Possibilities         0.801606
Screen Size                  0.763415
Grouser Type                 0.752378
Tupper Type                  0.752378
Pattern Changer              0.752213
Stick Length                 0.752213
Thumb                        0.752041
Screen Size.1                0.750626
Ripper                       0.740864
Driver System                0.740520
MachineHours CurrentMeter    0.642586
Control Type                 0.629928
Transmiss

In [ ]:
binary_categorical = [] # fill them with nans? doesnt work - dropped for now

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import r2_score

# --- 1. Load Your Processed Data ---
df = raw_data.copy()

# --- 2. Apply Your Best Encoding Strategy ---
# This is the 'simple' way that gave you R^2 = 0.895
print("Encoding all categorical columns...")
for col in df.select_dtypes(include=['object']).columns: # change into numerics
    df[col] = df[col].astype('category').cat.codes


X_train = df.drop('Sales Price', axis=1)
y_train = df['Sales Price']

# --- 4. Train the Model (First Pass) ---
print("Fitting model (Pass 1) to find useless features...")
xgb_reg = xgb.XGBRegressor(n_estimators=300, random_state=42, n_jobs=-1)
xgb_reg.fit(X_train, y_train)

# Get R^2 score to confirm
preds = xgb_reg.predict(X_train)
score_pass_1 = r2_score(y_train, preds)
print(f"R^2 Score (Pass 1): {score_pass_1:.5f}")

# --- 5. Find Features with Zero Importance ---
importances = xgb_reg.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Get the list of useless features
useless_features = importance_df[importance_df['Importance'] < 0.001]['Feature'].tolist()

if useless_features:
    print(f"\nFound {len(useless_features)} useless features (Importance < 0.001):")
    print(useless_features)
else:
    print("\nNo features found with zero importance.")

# --- 6. Test: Re-train Model WITHOUT Useless Features ---
if useless_features:
    print("\nRe-training model (Pass 2) after dropping useless features...")
    
    # Drop these features from both train and valid sets
    X_train_pruned = X_train.drop(columns=useless_features)
    
    xgb_reg_pruned = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    xgb_reg_pruned.fit(X_train_pruned, y_train)
    
    # Get the new score
    preds_pruned = xgb_reg_pruned.predict(X_train_pruned)
    score_pass_2 = r2_score(y_train, preds_pruned)
    
    print(f"R^2 Score (Pass 2, Pruned): {score_pass_2:.5f}")

Encoding all categorical columns...
Fitting model (Pass 1) to find useless features...
R^2 Score (Pass 1): 0.91603

Found 3 useless features (Importance < 0.001):
['Turbo Charged', 'Couple System', 'Grouser Tracks']

Re-training model (Pass 2) after dropping useless features...
R^2 Score (Pass 2, Pruned): 0.89570
